In [3]:
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer

In [4]:
#Now we will define a function which will compute and return the function value 
def evalf(x, n, lam):  
  #Input: x is a numpy array of size n 
  assert type(x) is np.ndarray  
  assert len(x) == n 
  return 0.5*(np.linalg.norm(np.matmul(A,x) - y))**2 + 0.5*lam*np.matmul(x.T,x)

In [5]:
def evalg(x, n, lam):
  assert type(x) is np.ndarray
  assert len(x) == n
  return lam*x + np.matmul(A.T, np.matmul(A, x) - y)

In [6]:
def evalh(x,n,lam):
  assert type(x) is np.ndarray  #do not allow arbitrary type arguments 
  assert len(x) == n #do not allow arbitrary size arguments 
  return lam*np.eye(n) + np.matmul(A.T, A)

In [7]:
#line search type 
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2
CONSTANT_STEP_LENGTH = 3

In [8]:
def compute_steplength_backtracking_scaled(x,n,lam, gradf, direction, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(x) == n
  assert type(gradf) is np.ndarray and len(gradf) == n
  #assert type(direction) is np.ndarray and len(direction) == 2  
   
  #assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
   
  #Complete the code 
  alpha = alpha_start
  gradf = evalg(x,n,lam)
  p=direction
  #np.matmul(np.matrix.transpose(gradf), p)
  while evalf(x+alpha*p,n,lam) > evalf(x,n,lam) + gamma*alpha*np.matmul(np.matrix.transpose(gradf), p) :
    alpha = rho*alpha 
  return alpha

In [9]:
import math

In [10]:
def find_minimizer_Newtonmethod(start_x, n,lam, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size n, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray #do not allow arbitrary type arguments 
  assert len(start_x) == n #do not allow arbitrary size arguments 
  assert type(tol) is float and tol>=0 
  
  x = start_x
  g_x = evalg(x,n,lam)
  h_x = evalh(x,n,lam)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  
  #print('iter:',k,  ' f(x):', evalf(x,n), ' gradient norm:', np.linalg.norm(g_x))
  x_newton =  []
  f_newton = []
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    #implement the Newton's method here
    D_k=np.linalg.inv(evalh(x,n,lam))
    direction = np.matmul(D_k,-g_x)
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled(x,n,lam,g_x, direction, alpha_start, rho, gamma)  
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 1.0
      
    else:  
      raise ValueError('Line search type unknown. Please check!')
    #x_newton.append(math.log(np.linalg.norm(x - x_bar)))
    #f_newton.append(math.log(np.linalg.norm(evalf(x,n,lam) - evalf(x_bar,n,lam))))
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x)))
    k += 1 #increment iteration
    g_x = evalg(x, n,lam) #compute gradient at new point
    
  return x, evalf(x,n,lam)

In [15]:
def find_minimizer_BFGS_method(start_x, n,lam, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size n, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray #do not allow arbitrary type arguments 
  assert len(start_x) == n #do not allow arbitrary size arguments 
  assert type(tol) is float and tol>=0 
  
  x = start_x
  x0 = x
  g_x = evalg(x,n,lam)
  g0 = g_x

  
  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  
  #print('iter:',k,  ' f(x):', evalf(x,n), ' gradient norm:', np.linalg.norm(g_x))
  x_bfgs = []
  f_bfgs = []
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    #implement the Newton's method here
    
    x0 = x
    g_x = evalg(x,n,lam)
    g0 = g_x

    if k==0:
      B_k=np.identity(n)
    else:

      I = np.identity(n)
      
      mu_k = 1/np.matmul(np.transpose(y_k),s_k)

      B_k = np.add(np.matmul(np.matmul(np.subtract(I, mu_k*np.outer( s_k, np.transpose(y_k))),B_k), np.subtract(I, mu_k*np.outer(y_k,np.transpose(s_k)))), mu_k*np.outer( s_k, np.transpose(s_k)))
    direction = np.matmul(B_k,-g_x)
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled(x,n,lam,g_x, direction, alpha_start, rho, gamma)     
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 1.0
    else:  
      raise ValueError('Line search type unknown. Please check!')
    #x_bfgs.append(math.log(np.linalg.norm(x0 - x_bar)))
    #f_bfgs.append(math.log(np.linalg.norm(evalf(x0,n,lam) - evalf(x_bar,n,lam))))
    x = np.subtract(x, np.multiply(step_length,np.matmul(B_k, g_x)))
    g_x = evalg(x, n,lam)
    s_k = x-x0 
    y_k=  g_x-g0
    k += 1 #increment iteration
    #g_x = evalg(x, n,lam) #compute gradient at new point
   
  return x, evalf(x,n,lam)

In [12]:
alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-5
lam=0.1

In [ ]:
#Code for Newton method
import numpy as np
import timeit
np.random.seed(1000) #for repeatability
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.1
eps = np.random.randn(N,1) #random noise
#For each value of dimension in the ds array, we will check the behavior of Newton method
for i in range(np.size(ds)):
  d=ds[i]
  A = np.random.randn(N,d)
  xorig = np.ones((d,1))
  my_start_x = np.zeros((d,1))
  y = np.dot(A,xorig) + eps
  start = timeit.default_timer()
  #call Newton method with A,y,lambda and obtain the optimal solution x_opt
  #x_opt = Newton(A,y,lambda_reg)
  x_opt, opt_fval = find_minimizer_Newtonmethod(my_start_x, d,lam, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
  newtontime = timeit.default_timer() - start #time is in seconds
  #print the total time and the L2 norm difference || x_opt - xorig|| for Newton method
  diff = np.subtract(x_opt, xorig)
  print("\n\nd:", d)
  print("Total time:", newtontime)
  print("L2 norm difference ||Ax* - y||_2^2: ", np.linalg.norm(np.subtract(np.matmul(A, x_opt), y))**2)
  print("L2 norm difference || x_opt - xorig||: ", np.linalg.norm(diff)**2)



d: 1000
Total time: 1.4444547960001728
L2 norm difference ||Ax* - y||_2^2:  0.0021220541764787873
L2 norm difference || x_opt - xorig||:  817.7202119606469


d: 5000
Total time: 112.31632143199977
L2 norm difference ||Ax* - y||_2^2:  0.00047397442649197916
L2 norm difference || x_opt - xorig||:  4774.913361486921


d: 10000
Total time: 962.785232056
L2 norm difference ||Ax* - y||_2^2:  0.00017517588089034528
L2 norm difference || x_opt - xorig||:  9825.861491699694


In [1]:
import pandas as pd
Dimention_of_A=[(200, 1000),(200,5000),(200,10000)]
Total_Time = [1.4444547960001728, 112.31632143199977, 962.785232056]
L2_norm_difference1= [ 0.0021220541764787873, 0.00047397442649197916,  0.00017517588089034528]
L2_norm_difference2= [817.7202119606469, 4774.913361486921 , 9825.861491699694]
data = {'Dimention_of_A ': Dimention_of_A,'Total Time':Total_Time,'||Ax^* - y||2^2':L2_norm_difference1,'||x^* - x{orig}||_2^2':L2_norm_difference2}
df = pd.DataFrame(data)
df

,Dimention_of_A,Total Time,||Ax^* - y||2^2,||x^* - x{orig}||_2^2
0,"(200, 1000)",1.444455,0.002122,817.720212
1,"(200, 5000)",112.316321,0.000474,4774.913361
2,"(200, 10000)",962.785232,0.000175,9825.861492


In [ ]:
#Code for BFGS method
import numpy as np
import timeit
np.random.seed(1000) #for repeatability
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.1
eps = np.random.randn(N,1) #random noise
#For each value of dimension in the ds array, we will check the behavior of BFGS method
for i in range(np.size(ds)):
  d=ds[i]
  A = np.random.randn(N,d)
  xorig = np.ones((d,1))
  my_start_x = np.zeros((d,1))
  y = np.dot(A,xorig) + eps
  start = timeit.default_timer()
  #call Newton method with A,y,lambda and obtain the optimal solution x_opt
  #x_opt = Newton(A,y,lambda_reg)
  newtontime = timeit.default_timer() - start #time is in seconds
  #print the total time, ||Ax_opt-y||^2 and the L2 norm difference || x_opt - xorig||^2 for Newton method  
  start = timeit.default_timer()
  #call BFGS method with A,y,lambda and obtain the optimal solution x_opt_bfgs
  #x_opt_bfgs = BFGS(A,y,lambda_reg)
  x_opt_bfgs, opt_fval_bfgs = find_minimizer_BFGS_method(my_start_x, d,lam,my_tol,BACKTRACKING_LINE_SEARCH,0.9, 0.5,0.5)
  bfgstime = timeit.default_timer() - start #time is in seconds
  #print the total time, ||Ax_opt_bfgs-y||^2 and the L2 norm difference || x_opt_bfgs - xorig||^2 for BFGS method
  diff = np.subtract(x_opt_bfgs, xorig)
  print("d:", d)
  print("Total time:", bfgstime)
  print("L2 norm difference ||Ax* - y||_2^2: ", np.linalg.norm(np.subtract(np.matmul(A, x_opt_bfgs), y))**2)
  print("L2 norm difference || x_opt - xorig||: ", np.linalg.norm(diff)**2)
  

d: 1000
Total time: 21.82778980799958
L2 norm difference ||Ax* - y||_2^2:  0.0021220537342639904
L2 norm difference || x_opt - xorig||:  817.7202119597779
d: 5000
Total time: 1778.1315902699998
L2 norm difference ||Ax* - y||_2^2:  0.0004739707628245362
L2 norm difference || x_opt - xorig||:  4774.913361486827


In [ ]:
import pandas as pd
Dimention_of_A=[(200, 1000),(200,5000)]
Total_Time = [21.82778980799958, 1778.1315902699998]
L2_norm_difference1= [ 0.0021220537342639904, 0.0004739707628245362]
L2_norm_difference2= [817.7202119597779, 4774.913361486827]
data2 = {'Dimention_of_A ': Dimention_of_A,'Total Time':Total_Time,'||Ax^* - y||2^2':L2_norm_difference1,'||x^* - x{orig}||_2^2':L2_norm_difference2}
df2 = pd.DataFrame(data2)
df2

,Dimention_of_A,Total Time,||Ax^* - y||2^2,||x^* - x{orig}||_2^2
0,"(200, 1000)",21.82779,0.002122,817.720212
1,"(200, 5000)",1778.13159,0.000474,4774.913361


**We are able to find the minimum using both the mathod regularized OLSLR and Direct OLSLR as hessian never becomes non-invertible (singular matrix) and we do run into trouble because of approximation of hessian. Reguralized OLSLR makes approximated hessian positive definite by adding lambda times identity and hence we are able to find the hessian inverse and then we find the minimum.**